In [2]:
# Step 1: Installation Commands (Run these FIRST in Colab or terminal)
"""
!pip install --upgrade --quiet qiskit>=2.0 qiskit-aer>=0.14

!pip install --quiet qiskit-algorithms>=1.0 qiskit-ibm-runtime>=0.23
!pip install --quiet cryptography>=42.0 numpy>=2.0 matplotlib>=3.9
!pip install --quiet pqcrypto>=0.2  # For real Kyber implementation
!pip install --quiet qiskit-experiments>=0.7  # For advanced quantum experiments
"""

In [3]:
# Minimal working imports for Kyber cryptography lab
import numpy as np
import hashlib
import matplotlib.pyplot as plt
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

# Essential Qiskit 2.0+ components
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

print("✅ Minimal imports for Kyber Quantum Cryptography Lab ready!")
print("Date: February 2026")
print("Environment: Qiskit 2.0+ with Post-Quantum Cryptography Focus")

# Version check
import sys
print(f"\nPython: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print(f"NumPy: {np.__version__}")

✅ Minimal imports for Kyber Quantum Cryptography Lab ready!
Date: February 2026
Environment: Qiskit 2.0+ with Post-Quantum Cryptography Focus

Python: 3.12.12
NumPy: 2.0.2


In [9]:
#!pip3 install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.0 MB/s eta 0:00:00


In [11]:
import hashlib
import secrets
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import numpy as np
import json

class QuantumSafeChat:
    def __init__(self, name):
        self.name = name
        self.q = 3329  # Kyber parameter
        self.n = 256   # Polynomial degree
        self.k = 3     # Security parameter
        self.public_key, self.private_key = self.make_keys()
        self.shared_secret = None

    def make_keys(self):
        """Generate Kyber-like key pair (simplified for demo)"""
        # In real Kyber: A is a k×k matrix of polynomials
        # For simplicity: A will be a k-length list of coefficients
        A = [[secrets.randbelow(self.q) for _ in range(self.n)] for _ in range(self.k)]
        s = [secrets.randbelow(3) - 1 for _ in range(self.n)]  # small coefficients: -1, 0, 1
        e = [secrets.randbelow(3) - 1 for _ in range(self.n)]  # small error

        # Public key: t = A * s + e (mod q)
        # Simplified: t = A[i] * s + e (mod q)
        t = []
        for i in range(self.k):
            # Simple polynomial multiplication (mod q)
            coeffs = [0] * self.n
            for j in range(self.n):
                coeffs[j] = (A[i][j] * s[j] + e[j]) % self.q
            t.append(coeffs)

        # Public key is (A, t), private key is s
        return {"A": A, "t": t}, s

    def generate_shared_secret(self, other_public_key):
        """Generate shared secret using other party's public key"""
        A = other_public_key["A"]
        t = other_public_key["t"]

        # Generate ephemeral secret
        s_prime = [secrets.randbelow(3) - 1 for _ in range(self.n)]
        e_prime = [secrets.randbelow(3) - 1 for _ in range(self.n)]

        # Compute u = A^T * s' + e'
        u = []
        for i in range(self.k):
            coeffs = [0] * self.n
            for j in range(self.n):
                coeffs[j] = (A[i][j] * s_prime[j] + e_prime[j]) % self.q
            u.append(coeffs)

        # Compute v = t^T * s' + e''
        e_double_prime = [secrets.randbelow(3) - 1 for _ in range(self.n)]
        v = [0] * self.n
        for j in range(self.n):
            v[j] = (t[0][j] * s_prime[j] + e_double_prime[j]) % self.q

        # Derive shared secret from v (simplified)
        shared_bytes = self._hash_vector(v)

        # Use KDF to get AES key
        self.shared_secret = hashlib.shake_256(shared_bytes).digest(32)

        # Return ciphertext (u, v) for the other party
        return {"u": u, "v": v}

    def derive_shared_secret(self, ciphertext, my_private_key):
        """Derive shared secret from ciphertext using my private key"""
        u = ciphertext["u"]
        v = ciphertext["v"]
        s = my_private_key

        # Compute shared value: w = v - s^T * u
        w = [0] * self.n
        for j in range(self.n):
            w[j] = (v[j] - u[0][j] * s[j]) % self.q

        # Derive shared secret
        shared_bytes = self._hash_vector(w)
        self.shared_secret = hashlib.shake_256(shared_bytes).digest(32)
        return self.shared_secret

    def _hash_vector(self, vector):
        """Hash a vector to bytes"""
        vector_bytes = b''.join([x.to_bytes(2, 'big') for x in vector])
        return hashlib.sha256(vector_bytes).digest()

    def encrypt_message(self, message):
        """Encrypt a message using the shared secret"""
        if not self.shared_secret:
            raise ValueError("No shared secret established!")

        # Use AES in GCM mode for authenticated encryption
        iv = secrets.token_bytes(12)
        cipher = AES.new(self.shared_secret, AES.MODE_GCM, nonce=iv)

        message_bytes = message.encode('utf-8')
        ciphertext, tag = cipher.encrypt_and_digest(pad(message_bytes, AES.block_size))

        # Return IV + tag + ciphertext
        return iv + tag + ciphertext

    def decrypt_message(self, encrypted_data):
        """Decrypt a message using the shared secret"""
        if not self.shared_secret:
            raise ValueError("No shared secret established!")

        # Split the encrypted data
        iv = encrypted_data[:12]
        tag = encrypted_data[12:28]
        ciphertext = encrypted_data[28:]

        # Decrypt
        cipher = AES.new(self.shared_secret, AES.MODE_GCM, nonce=iv)
        try:
            decrypted_bytes = cipher.decrypt_and_verify(ciphertext, tag)
            message = unpad(decrypted_bytes, AES.block_size).decode('utf-8')
            return message
        except (ValueError, KeyError) as e:
            return f"Decryption failed: {str(e)}"

    def get_public_key(self):
        """Return public key for sharing"""
        return self.public_key

    def chat(self, message, recipient):
        """Send an encrypted message to another chat participant"""
        if recipient.shared_secret is None:
            # Establish shared secret first
            ciphertext = recipient.generate_shared_secret(self.public_key)
            self.derive_shared_secret(ciphertext, self.private_key)

        encrypted = self.encrypt_message(message)
        decrypted = recipient.decrypt_message(encrypted)

        print(f"{self.name} -> {recipient.name}: {message}")
        print(f"Encrypted: {encrypted.hex()[:50]}...")
        print(f"Decrypted by {recipient.name}: {decrypted}\n")

# Create two chat participants
alice = QuantumSafeChat("Alice")
bob = QuantumSafeChat("Bob")

# Establish shared secret (simulating key exchange)
print("=== Establishing Shared Secret ===")
ciphertext = bob.generate_shared_secret(alice.get_public_key())
alice.derive_shared_secret(ciphertext, alice.private_key)
bob.derive_shared_secret(ciphertext, bob.private_key)

print(f"Alice's shared secret: {alice.shared_secret.hex()[:20]}...")
print(f"Bob's shared secret: {bob.shared_secret.hex()[:20]}...")
print(f"Secrets match: {alice.shared_secret == bob.shared_secret}\n")

# Chat messages
print("=== Secure Chat ===")
alice.chat("Hello Bob, how are you?", bob)
bob.chat("I'm good Alice! How's the quantum resistance?", alice)
alice.chat("Our chat is quantum-safe with Kyber-like encryption!", bob)

# Demo different message
print("=== Another Message ===")
encrypted_msg = alice.encrypt_message("This is a secret quantum-safe message!")
print(f"Encrypted message: {encrypted_msg.hex()[:50]}...")
decrypted_msg = bob.decrypt_message(encrypted_msg)
print(f"Decrypted message: {decrypted_msg}")

=== Establishing Shared Secret ===
Alice's shared secret: ddaa9297ade18c9622f8...
Bob's shared secret: a7f80506eef4c402a6af...
Secrets match: False

=== Secure Chat ===
Alice -> Bob: Hello Bob, how are you?
Encrypted: abd21b0313ad6f86f060e43266796cad8d586fb5271e6b576f...
Decrypted by Bob: Decryption failed: MAC check failed

Bob -> Alice: I'm good Alice! How's the quantum resistance?
Encrypted: 350ce8752b736e0be37505923e13653014f352b673cd540a6a...
Decrypted by Alice: Decryption failed: MAC check failed

Alice -> Bob: Our chat is quantum-safe with Kyber-like encryption!
Encrypted: fbd97c441e099ef912ac5785adcc98fa568738ae69d6182ed6...
Decrypted by Bob: Decryption failed: MAC check failed

=== Another Message ===
Encrypted message: 5418831328b850b50f07613382faa1cfa10e420b67b444fc7a...
Decrypted message: Decryption failed: MAC check failed
